In [1]:
import pickle

import ipywidgets as widgets
import pandas as pd
import pingouin as pg

from tse_datatools.data.workspace import Workspace
from tse_analytics.data.data_hub import DataHub
from tse_analytics.messaging.messenger import Messenger

## Initialization

In [2]:
messenger = Messenger()
data_hub = DataHub(messenger)

## Load TSE Analytics workspace

In [3]:
path = r"C:\Users\anton\OneDrive - TSE Systems, Inc\Desktop\Merged.workspace"

with open(path, "rb") as file:
    workspace = pickle.load(file)

## List available datasets

In [4]:
datasets_widget = widgets.Select(
    options=[dataset.name for dataset in workspace.datasets],
    description='Datasets:',
    layout=widgets.Layout(height='150px'),
)
display(datasets_widget)

Select(description='Datasets:', layout=Layout(height='150px'), options=('calo_#38197_20230113_lot1', 'calo_#38…

## Select active dataset

In [10]:
for d in workspace.datasets:
    if d.name == datasets_widget.value:
        dataset = d

dataset.original_df.info()

data_hub.set_selected_dataset(dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 53 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   DateTime   90000 non-null  datetime64[ns] 
 1   Timedelta  90000 non-null  timedelta64[ns]
 2   Bin        90000 non-null  category       
 3   Animal     90000 non-null  category       
 4   Box        90000 non-null  category       
 5   Run        90000 non-null  category       
 6   TempL      90000 non-null  float64        
 7   HumL       90000 non-null  float64        
 8   LightL     90000 non-null  float64        
 9   PressL     90000 non-null  int64          
 10  CO2L       90000 non-null  int64          
 11  NoiseL     90000 non-null  int64          
 12  MotionL    90000 non-null  int64          
 13  Ref.SFlow  90000 non-null  float64        
 14  Ref.O2     90000 non-null  float64        
 15  Ref.CO2    90000 non-null  float64        
 16  Flow       90000 non-n

In [11]:
animals_widget = widgets.SelectMultiple(
    options=[animal.id for animal in dataset.animals.values()],
    description='Animals:',
    layout=widgets.Layout(height='300px'),
)
display(animals_widget)

SelectMultiple(description='Animals:', layout=Layout(height='300px'), options=(1, 7, 2, 8, 3, 5, 4, 6, 15, 9, …

In [12]:
selected_animals = [animal for animal in dataset.animals.values() if animal.id in animals_widget.value]
data_hub.set_selected_animals(selected_animals)

In [13]:
factors_widget = widgets.Select(
    options=[factor.name for factor in dataset.factors.values()],
    description='Factors:',
)
display(factors_widget)

Select(description='Factors:', options=('f1', 'f2', 'sex'), value='f1')

In [14]:
selected_factor = dataset.factors[factors_widget.value]
data_hub.set_selected_factor(selected_factor)

In [15]:
variable_widget = widgets.Dropdown(
    options=[variable.name for variable in dataset.variables.values()],
    description='Variable:',
)
display(variable_widget)

Dropdown(description='Variable:', options=('CenA', 'CenF', 'CenT', 'CO2', 'CO2L', 'dCO2', 'DistD', 'DistK', 'd…

In [16]:
selected_variable = variable_widget.value
df = data_hub.get_current_df(calculate_error=False, variables=[selected_variable])
anova = pg.anova(data=df, dv=selected_variable, between=selected_factor.name, detailed=True)
print(anova)

   Source           SS     DF         MS            F  p-unc       np2
0      f1    27.478678      1  27.478678  2198.695006    0.0  0.026057
1  Within  1027.087742  82182   0.012498          NaN    NaN       NaN
